# Notebook de pruebas en el manejo de series temporales con Pandas

## 1. Importar liberías

In [1]:
import pandas
pandas.__version__

'1.0.5'

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import os

In [5]:
from datetime import datetime, timedelta

In [6]:
from tqdm import tqdm

In [7]:
import glob

In [8]:
import fileinput
import sys, glob

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt

## 2 Datos
### 2.1 Definir rutas de trabajo

In [10]:
rutaDatos = 'E:/Ejercicios_AEMet/Historicos_Est_TPVHR/'

In [11]:
rutaOut = 'E:/Ejercicios_AEMet/Historicos_Est_TPVHR/Solucion_ejercicio/'

In [12]:
parametros = pd.DataFrame(columns = ['INDICATIVO','AÑO','MES','DÍA','HORA','MINUTO','PREC','TA','VV10M','DV10M','HR','RAGLOB'])


In [13]:
parametros

,INDICATIVO,AÑO,MES,DÍA,HORA,MINUTO,PREC,TA,VV10M,DV10M,HR,RAGLOB


In [14]:
carpeta_trabajo = os.listdir(rutaDatos)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'E:/Ejercicios_AEMet/Historicos_Est_TPVHR/'

In [ ]:
carpeta_trabajo

### 2.2 Leer series de precipitación de 2010
Importar el archivo _2010.txt_ y extraer los datos de precipitación. A partir de esos datos, crear un _data frame_ en el que las filas sean los pasos temporales (frecuencia 10 min) y las columnas las estaciones.

In [ ]:
file2010 = carpeta_trabajo[1]
file2010

In [ ]:
df = pd.DataFrame(data=parametros)

In [ ]:
datos_prec = pd.read_csv(rutaDatos + file2010, sep=';', decimal=',', usecols=['INDICATIVO', 'PREC'], encoding='latin1')
datos_prec.head()

In [ ]:
datesStr = datos_prec.AÑO.astype(str) + '-' + datos_prec.MES.astype(str) + '-' + datos_prec.DIA.astype(str) + ' ' + datos_prec.HORA.astype(str) + ':' + datos_prec.MINUTO.astype(str)
datos_prec.index = pd.to_datetime(datesStr, format='%Y-%m-%d %H:%M')

In [ ]:
datos_prec.head()

In [ ]:
start, end = datos_prec.index.min(), datos_prec.index.max()
start, end

In [ ]:
dates = pd.date_range(start, end, freq='10min')

In [ ]:
estaciones = datos_prec.INDICATIVO.unique()

In [ ]:
df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')

df.head()

In [ ]:
stn = '1014A'

In [ ]:
for stn in estaciones:
    aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']
    df[stn] = aux

In [ ]:
aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']

In [ ]:
aux.head()

In [ ]:
df[stn] = aux

In [ ]:
df

In [ ]:
csv_files = glob.glob('*.csv')

In [ ]:
print (csv_files)

In [ ]:
list_data = []

for file in csv_files:
    data = pd.read_csv(file)
    list_data.append(data)

In [ ]:
list_data

In [ ]:
def resultados(fileIn, variable, pathOut=None):
       
    carpeta_trabajo = os.listdir(rutaDatos)
    
    #Se lee el archivo csv necesario y se guarda en una tabla que se denominará 'datos_var'
    datos_var = pd.read_csv(fileIn, sep=';', decimal=',', usecols=['INDICATIVO', 'AÑO', 'MES', 'DIA', 'HORA', 'MINUTO', variable], encoding='latin1')
    
    
    datesStr = datos_var.AÑO.astype(str) + '-' + datos_var.MES.astype(str) + '-' + datos_var.DIA.astype(str) + ' ' + datos_var.HORA.astype(str) + ':' + datos_var.MINUTO.astype(str)
    datos_var.index = pd.to_datetime(datesStr, format='%Y-%m-%d %H:%M')
    
#     Se define el inicio y final
    start, end = datos_var.index.min(), datos_var.index.max()
    
    # Se define también el paso temporal, en este caso serán 10 minutos
    dates = pd.date_range(start, end, freq='10min')
    # leer estaciones
    estaciones = datos_var.INDICATIVO.unique()
    
    # crear dataframe vacío
    df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')
    df.index.name = 'FECHA'
    
    # Se crea un bucle cuya misión será recorrer todas las estaciones existentes en el fichero, y asociar sus valores de 
    # las variables correspondientes
    for stn in estaciones:
        aux = datos_var.loc[datos_var.INDICATIVO == stn, variable]
        df[stn] = aux
        
    #escribir archivo de salida
    if pathOut is not None:
        fileOut = pathOut + fileIn.split('/')[-1].split('.')[0] + variable + '.csv'
        df.to_csv(fileOut)
    
    return df

In [ ]:
df = resultados(rutaDatos + '2010_.txt', variable='PREC', pathOut=rutaOut)

In [ ]:
df.head()

In [ ]:
df.index.name = 'FECHA'

In [ ]:
df.head()

In [ ]:
rutaOut + 'test.csv'

In [ ]:
datos_prec.to_csv(rutaOut + 'test.csv')

In [ ]:
carpeta_trabajo[1]

In [ ]:
file = carpeta_trabajo[1]

In [ ]:
precipitacion(file)

### 2.3 Leer datos de precipitación de todos los años
En base al código del punto anterior, hacer un bucle para leer el archivo de cada año. El resultado final ha de ser un _data frame_ en el que las filas representan todos los pasos temporales desde 2009 a 2020 y las columnas los códigos de las estaciones.

In [ ]:
rutaDatos
nombres_archivos = os.listdir(rutaDatos)

In [ ]:
nombres_archivos

In [ ]:
files = []
for file in nombres_archivos:
    if file.endswith('_.txt'):
        files.append(file)

In [ ]:
files = [file for file in nombres_archivos if file.endswith('_.txt')]

In [ ]:
P = pd.DataFrame(dtype='float64')
for file in files:
    aux = resultados(rutaDatos + file, 'PREC')
    P = pd.concat((P, aux), axis=0)

In [ ]:
df.to_csv(rutaOut + 'Pruebas1.csv')

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
files

In [ ]:
# def buscar_archivos(ruta):
#     archivos_texto = []
#     archivos = os.listdir(rutaDatos)
#     for archivo in archivos:
#         if archivo[-5:] == '_.txt':
#             archivos_texto.append(archivo)
#     return archivos_texto

In [ ]:
# buscar_archivos(rutaDatos)

In [ ]:
for filename in nombres_archivos:
    if filename.endswith('_.txt'):
        f = open(filename)
        lines = f.read()
        print (lines)
        continue
    else:
continue
    

In [ ]:
datesStr = datos_prec.AÑO.astype(str) + '-' + datos_prec.MES.astype(str) + '-' + datos_prec.DIA.astype(str) + ' ' + datos_prec.HORA.astype(str) + ':' + datos_prec.MINUTO.astype(str)
datos_prec.index = pd.to_datetime(datesStr, format='%Y-%m-%d %H:%M')

In [ ]:
datos_prec.head()

In [ ]:
start, end = datos_prec.index.min(), datos_prec.index.max()
start, end

In [ ]:
dates = pd.date_range(start, end, freq='10min')

In [ ]:
dates

In [ ]:
estaciones = datos_prec.INDICATIVO.unique()
estaciones

In [ ]:
df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')
df

In [ ]:
for stn in estaciones:
    aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']
    df[stn] = aux

In [ ]:
aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']

In [ ]:
aux

In [ ]:
df[stn] = aux
df

### 2.4 Leer datos de temperatura de todos los años
Repetir el código anterior, pero extrayendo datos de temperatura en vez de precipitación.

In [ ]:
rutaDatos

In [ ]:
file2010 = carpeta_trabajo[1]

In [ ]:
parametros

In [ ]:
files = []
for file in nombres_archivos:
    if file.endswith('_.txt'):
        files.append(file)

In [ ]:
datos_temp = pd.read_csv(rutaDatos + file2010, sep=';', decimal=',', usecols=['INDICATIVO', 'AÑO', 'MES', 'DIA', 'HORA', 'MINUTO', 'TA'], encoding='latin1')
datos_temp.head()

In [ ]:
datesStr = datos_temp.AÑO.astype(str) + '-' + datos_temp.MES.astype(str) + '-' + datos_temp.DIA.astype(str) + ' ' + datos_temp.HORA.astype(str) + ':' + datos_temp.MINUTO.astype(str)
datos_temp.index = pd.to_datetime(datesStr, format='%Y-%m-%d %H:%M')

In [ ]:
end, start = datos_temp.index.max(), datos_temp.index.min()

In [ ]:
dates = pd.date_range(start, end, freq='10min')

In [ ]:
estaciones = datos_temp.INDICATIVO.unique()

In [ ]:
df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')
df.head()

In [ ]:
stn = '1111X'

In [ ]:
aux = datos_temp.loc[datos_temp.INDICATIVO == stn, 'TA']
aux

In [ ]:
aux.head()

In [ ]:
df[stn] = aux

In [ ]:
df

In [ ]:
for stn in estaciones:
    aux = datos_temp.loc[datos_temp.INDICATIVO == stn, 'TA']
    df[stn] = aux

In [ ]:
df.to_csv(rutaOut + 'temperatura.csv')

In [ ]:
df.index.name = 'FECHA'

***

In [ ]:
files = []
for file in nombres_archivos:
    if file.endswith('_.txt'):
        files.append(file)
files

In [ ]:
T = pd.DataFrame(dtype='float64')
for file in files:
    aux = resultados(rutaDatos + file, 'TA')
    T = pd.concat((T, aux), axis=0)

In [ ]:
T.shape

In [ ]:
rutaOut + 'Temperatura.csv'

In [ ]:
T.to_csv(rutaOut + 'Temperatura.csv')

### 2.5 Visualización de los datos
#### Serie de temperatura de la estación de Oviedo
Hacer un gráfico de línea con la serie de temperatura de la estación de Oviedo. Mostrar también la serie con la temperatura media entre todas las estaciones.

In [ ]:
datos_temp.INDICATIVO

In [ ]:
temp_Oviedo = df['1249X']

In [ ]:
temp_Oviedo.head()

In [ ]:
T_Oviedo = pd.DataFrame(dtype='float64')
for file in files:
    aux_Oviedo = resultados(rutaDatos + file, 'TA')['1249X']
    T_Oviedo = pd.concat((T_Oviedo, aux_Oviedo), axis=0)

In [ ]:
T_Oviedo

Calculo de la temperatura media de todas las estaciones

In [ ]:
df.Media = df.mean(axis='columns').round(1)

In [ ]:
df.Media, T_Oviedo

In [ ]:
T = pd.DataFrame(dtype='float64')
for file in files:
    aux = resultados(rutaDatos + file, 'TA')
    T = pd.concat((T, aux), axis=0)

In [ ]:
T.insert(6, 'Media', True)

In [ ]:
T.Media = T.mean(axis='columns').round(1)

In [ ]:
T.Media.plot()

In [ ]:
T_Oviedo.plot()

***

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(T['1249X'], c='red', lw=1, label='Oviedo')
plt.plot(T.Media, c='k', lw=1, label='media')
plt.xlabel('fecha')
plt.xlim(datetime(2014, 1, 1), datetime(2014, 1, 31))
plt.ylabel('T (°C)')
plt.legend();

#### Series de temperatura de todas las estaciones
En base al código anterior, crear una figura con tantos gráficos como estaciones con datos de temperatura. Al igual que antes, en cada gráfico se muestra la serie de temperatura de la estación y la serie media entre todas las estaciones.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16, 12), sharex=True, sharey=True)

axes[0,0].set_xlim(datetime(2014, 1, 1), datetime(2014, 1, 31))
axes[0,0].plot(T['1249X'], c='red', lw=1, label='Oviedo')
axes[0,0].plot(T.Media, c='k', lw=1, label='media')
axes[0,0].legend()

axes[0,1].plot(T['1111X'], c='red', lw=1, label='Santander')
axes[0,1].plot(T.Media, c='k', lw=1, label='media')
axes[0,1].legend();

axes[1,0].plot(T['1208H'], c='red', lw=1, label='Gijón')
axes[1,0].plot(T.Media, c='k', lw=1, label='media')
axes[1,0].legend();

axes[1,1].plot(T['9091R'], c='red', lw=1, label='Vitoria')
axes[1,1].plot(T.Media, c='k', lw=1, label='media')
axes[1,1].legend();

axes[2,0].plot(T['1014A'], c='red', lw=1, label='San Sebastián')
axes[2,0].plot(T.Media, c='k', lw=1, label='media')
axes[2,0].legend();

axes[2,1].plot(T['1505'], c='red', lw=1, label='Lugo')
axes[2,1].plot(T.Media, c='k', lw=1, label='media')
axes[2,1].legend();

In [ ]:
T_SanSebastian = T['1014A']
T_Oviedo = T['1249X']
T_Vitoria = T['9091R']
T_Santander = T['1111X']
T_Lugo = T['1505']
T_Gijon = T['1208H']

In [ ]:
T_Gijon.plot()
T_Lugo.plot()
T_Oviedo.plot()
T_SanSebastian.plot()
T_Santander.plot()
T_Vitoria.plot()

In [ ]:
T_Gijon.plot()
T.Media.plot()

## 3 Tratamiento de datos
### 3.1 Serie de precipitación diaria
Calcular la serie de precipitación diaria acumulada de todas las estaciones.

In [ ]:
datos_prec = pd.read_csv(rutaDatos + file2010, sep=';', decimal=',', usecols=['INDICATIVO', 'AÑO', 'MES', 'DIA', 'HORA', 'MINUTO', 'PREC'], encoding='latin1')

In [ ]:
datesStr = datos_prec.AÑO.astype(str) + '-' + datos_prec.MES.astype(str) + '-' + datos_prec.DIA.astype(str) + ' ' + datos_prec.HORA.astype(str) + ':' + datos_prec.MINUTO.astype(str)
datos_prec.index = pd.to_datetime(datesStr, format='%Y-%m-%d %H:%M')

In [ ]:
datos_prec.head()

In [ ]:
start, end = datos_prec.index.min(), datos_prec.index.max()
start, end

In [ ]:
dates = pd.date_range(start, end, freq='10min')

In [ ]:
estaciones = datos_prec.INDICATIVO.unique()

In [ ]:
df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')

In [ ]:
for stn in estaciones:
    aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']
    df[stn] = aux

In [ ]:
aux = datos_prec.loc[datos_prec.INDICATIVO == stn, 'PREC']

In [ ]:
df.head()

In [ ]:
df.insert(6, 'Acum', True)

In [ ]:
df

In [ ]:
df.Acum = df.sum(axis='columns')

In [ ]:
df

In [ ]:
P = pd.DataFrame(dtype='float64')
for file in files:
    aux = resultados(rutaDatos + file, 'PREC')
    P = pd.concat((P, aux), axis=0)

In [ ]:
P.Acum = np.nan

In [ ]:
P.head()

In [ ]:
plt.plot(P.sum(axis=1).round(1))

In [ ]:
Prec_acum_diaria = P.resample('1D').sum()

In [ ]:
Prec_acum_diaria.head()

In [ ]:
Prec_acum_diaria_TOTAL = Prec_acum_diaria.sum(axis=1)

In [ ]:
Prec_acum_diaria_TOTAL

### 3.2 Serie de precipitación mensual
Calcular la serie de precipitación mensual de todas las estaciones.

In [ ]:
Prec_acum_mensual = P.resample('1M').sum()

In [ ]:
Prec_acum_mensual.Acum = Prec_acum_mensual.sum(axis=1)

In [ ]:
Prec_acum_mensual

In [ ]:
Prec_acum_mensual[['1014A']]

In [ ]:
Prec_acum_mensual.Acum = Prec_acum.sum(axis=1)

In [ ]:
Prec_acum_mensual

In [ ]:
Prec_acum_mensual.drop('Total', axis=1)

Crear un gráfico de cajas con los datos mensuales.

In [ ]:
plt.figure(figsize=(18, 6))
plt.boxplot(Prec_acum_mensual.groupby(by=Prec_acum_mensual.month))
plt.show()

### 3.3 Serie de precipitación anual
Calcular la serie de precipitación anual de todas las estaciones.

In [ ]:
P.plot()

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(P)

In [ ]:
P[['1014A']], P[['1111X']], P[['1208H']], P[['1249X']], P[['1505']], P[['9091R']]

In [ ]:
P_1014A = P[['1014A']].resample('1Y').sum()

In [ ]:
P_1014A.rename(columns={'1014A':'San Sebastián(1014A)'})

In [ ]:
P_1111X = P[['1111X']].resample('1Y').sum()

In [ ]:
P_1111X.rename(columns={'1111X':'Santander (1111X)'})

In [ ]:
P_1208H = P[['1208H']].resample('1Y').sum()

In [ ]:
P_1208H.rename(columns={'1208H':'Gijón (1208H)'})

In [ ]:
P_1505 = P[['1505']].resample('1Y').sum()

In [ ]:
P_1505.rename(columns={'1505':'Lugo (1505)'})

In [ ]:
P_9091R = P[['9091R']].resample('1Y').sum()

In [ ]:
P_9091R.rename(columns={'9091R':'Vitoria (9091R)'})

In [ ]:
P_1249X = P[['1249X']].resample('1Y').sum()

In [ ]:
P[['1249X']].rename(columns={'1249X':'Oviedo (1249X)'})

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(P_1014A, c='red', lw=1, label='1014A')
plt.xlabel('fecha')
# plt.xlim(datetime(2014, 1, 1), datetime(2014, 1, 31))
plt.ylabel('P (mm)')
plt.legend();

In [ ]:
P_1014A = P[['1014A']].resample('1Y').sum()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(P_1014A, c='red', lw=1, label='1014A')
plt.plot(P_1111X, c='blue', lw=1, label='1111X')
plt.plot(P_1249X, c='green', lw=1, label='1249X')
plt.plot(P_9091R, c='orange', lw=1, label='9091R')
plt.plot(P_1208H, c='black', lw=1, label='1208H')
plt.plot(P_1505, c='plum', lw=1, label='1505')
plt.xlabel('fecha')
# plt.xlim(datetime(2014, 1, 1), datetime(2014, 1, 31))
plt.ylabel('P (mm)')
plt.legend();

plt.savefig('Precipitación anual.png', dpi=300, bbox_tight=True)

Crear un gráfico de dispersión de la precipitación media anual frente a la altitud de la estación.

In [ ]:
P[['1014A']]

### 3.4 Series de temperatura media, mínima y máxima diaria
Además de generar la serie, crear, para una estación, un gráfico con la temperatura mínima, media y máxima.

In [ ]:
Td_max = T.resample('1d').max()
Td_min = T.resample('1d').min()
Td_med = T.resample('1d').sum() / T.resample('1d').count()

In [ ]:
plt.plot(Td_max['1249X'])
plt.plot(Td_min['1249X'])
plt.plot(Td_med['1249X']);

In [ ]:
datos_temp = pd.read_csv(rutaDatos + file2010, sep=';', decimal=',', usecols=['INDICATIVO', 'TA'], encoding='latin1')

In [ ]:
estaciones = datos_temp.INDICATIVO.unique()

In [ ]:
df = pd.DataFrame(index=dates, columns=estaciones, dtype='float64')

df.head()

In [ ]:
T = pd.DataFrame(dtype='float64')
for file in files:
    aux = resultados(rutaDatos + file, 'TA')
    T = pd.concat((T, aux), axis=0)

In [ ]:
datos_temp

In [ ]:
temp_Oviedo = df['1249X']

In [ ]:
plt.plot(df['1249X'])

In [ ]:
temp_Oviedo.head()

In [ ]:
temp_Oviedo.resample('1Y').sum()

### 3.5 Serie de temperatura media mensual

A partir de la temperatura media mensual y la precipitación media mensual, crear un gráfico con el climograma de una estación.

## 4 Exportar resultados

### 4.1 Serie diaria de precipitación
Exportar la serie diaria de precipitación como archivo csv.

In [ ]:
year, month, day = 2020, 12, 28

In [ ]:
print('La fecha de hoy es ' + str(day) + '-' + str(month) + '-' + str(year))

In [ ]:
print('La fecha de hoy es {0}-{1}-{2}'.format(day, month, year))

In [ ]:
NSE = 0.55555555555

In [ ]:
print('NSE = {0:.3f}'.format(NSE))